# **Hand Gesture Classification Based on EMG Readings on Muscle Activity**
## Implementing *Gaussian Process Regression*, *Logistic Regression*, and *k-NN Classification*



### Dataset Description ([Link to Data](https://www.kaggle.com/datasets/kyr7plus/emg-4/data))

Each dataset line consists of 8 consecutive readings from all 8 sensors, resulting in 64 columns of EMG data. The final column in each line represents the gesture class that was performed while recording the data. The structure of each line is as follows:

    [8sensors][8sensors][8sensors][8sensors][8sensors][8sensors][8sensors][8sensors][GESTURE_CLASS]

### Data Recording Details

- **Frequency**: 200 Hz
- **Record Time per Line**: $ \frac{8}{200} $ seconds = 40 ms

### Gesture Classes

The gesture classes and their corresponding labels are:

- **Rock**: 0
- **Scissors**: 1
- **Paper**: 2
- **OK**: 3

**Gesture Descriptions**:
- **Rock, Paper, Scissors**: As in the traditional game.
- **OK**: Index finger touching the thumb, with the rest of the fingers spread.

### Data Collection

Each gesture was recorded 6 times for a duration of 20 seconds per recording. **Recording sessions started and ended with the gesture being held in a fixed position. This resulted in a total of 120 seconds of data for each gesture. All recordings were performed on the same right forearm within a short timespan.**

### Data Files

The recordings for each gesture class were concatenated into separate CSV files named according to their gesture class (0-3).


In [60]:
import numpy as np
import pandas as pd
import sys

from fvgp import GP
from fvgp.gp_kernels import matern_kernel_diff2

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

from loguru import logger

In [76]:
# To keep track of the training processes
logger.enable("fvgp")
logger.add(sys.stdout, filter="fvgp", level="INFO")

### Data Pre-Processing
For Binary Classification, only the first TWO gestures are used (i.e. only rock and scissors).

In [2]:
# Load data from csv files and separate features and labels
gesture_0 = pd.read_csv('0.csv').values
gesture_1 = pd.read_csv('1.csv').values

# Extract features and labels – only gesture 0 and 1 for binary classification.
X_0, y_0 = gesture_0[:, :-1], gesture_0[:, -1]
X_1, y_1 = gesture_1[:, :-1], gesture_1[:, -1]

# Concatenate data and labels
X = np.vstack((X_0, X_1))
y = np.concatenate((y_0, y_1))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Gaussian Process
Both an isotropic and anisotropic twice-differentiable Matérn kernel function are defined, marginally building on the off-the-shelf kernels provided within the `fvgp.gp_kernels` class. A length scale of 9 is initialized. The isotropic kernel was utilized, though one may expect results with the latter. The prior mean is not declared, so the model assumes it to be the (constant) mean of the `y_train` values.

In [89]:
def matern_kernel_isotropic(x1, x2, hps): # for one length_scale hyperparameter for each feature
    distance = np.sqrt(((x1[:, np.newaxis, :] - x2[np.newaxis, :, :]) ** 2).sum(axis=2))
    return matern_kernel_diff2(distance, hps[0])

def matern_kernel_anisotropic(x1, x2, length_scales): # for an individual length_scale hyperparameter for each unique feature
    length_scales = np.array(length_scales)
    d = np.sqrt(np.sum(((x1[:, np.newaxis, :] - x2[np.newaxis, :, :]) / length_scales) ** 2, axis=2))
    return matern_kernel_diff2(d, 1.0)

In [4]:
# Initialize the GP model for classification
gp_model = GP( 
    X_train_scaled,
    y_train,
    init_hyperparameters=np.array([9.0]),  # =np.ones(X_train_scaled.shape[1])*9 if the Anisotropic kernel is being used instead
    gp_kernel_function=matern_kernel_isotropic, # =matern_kernel_anisotropic
    noise_variances=np.ones(y_train.shape) * 0.01  # Assuming small noise variance (measurement error)
)

hps_bounds = np.array([[3, 20]])  

#Train using local optimization
gp_model.train(
    hyperparameter_bounds=hps_bounds,
    method='local',
    max_iter=30,
    tolerance=1,
)

In [77]:
# Classify the test set
posterior = gp_model.posterior_mean(X_test_scaled)
f_cov = gp_model.posterior_covariance(X_test_scaled)
f_var = f_cov['v(x)']  # Variances at the input points
f_cov_matrix = f_cov['S']  # Full posterior covariance matrix
f_mean = posterior["f(x)"] # Posterior means

gp_y_pred = f_mean.round()
gp_y_pred[gp_y_pred == -1] = 0 # truncating anomalies
gp_y_pred[gp_y_pred == 2] = 1

2024-07-20 14:41:04.276 | DEBUG    | fvgp.gp_lin_alg:calculate_Chol_solve:47 - calculate_Chol_solve


### Logistic Regression


In [78]:
lg_model = LogisticRegression() 
lg_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
lg_y_pred = logistic_model.predict(X_test_scaled)

### k-NN Classification
Optimizing the number of (odd) neighbours with a 5-fold cross validation.

In [79]:
# Define the parameter grid for the number of neighbors
knn_param_grid = {'n_neighbors': range(3, 21, 2)}

# Initialize the GridSearchCV with the KNeighborsClassifier
grid_search = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)
best_n_neighbors = grid_search.best_params_['n_neighbors']

# Train the final model with the optimal number of neighbors
knn_model = KNeighborsClassifier(n_neighbors=best_n_neighbors)
knn_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
knn_y_pred = knn_model.predict(X_test_scaled)

## Accuracy and Results

The posterior means of the Gaussian processe attains a near score, 98.11%, when trained with the 'default' kernel and prior mean, as compared to 60.71% with the Sigmoid function, and 84.35% with the optimal 3-NN Classifier.

In [81]:
gp_accuracy = accuracy_score(y_test, gp_y_pred)
lg_accuracy = accuracy_score(y_test, lg_y_pred)
knn_accuracy = accuracy_score(y_test, knn_y_pred)
print(f'Gaussian Process Classification Test Set Accuracy: {gp_accuracy:.4f}\n')
print(f'Logistic Regression Test set accuracy: {lg_accuracy:.4f}\n')
print(f'Optimal ({best_n_neighbors}) NN Test set accuracy: {knn_accuracy:.4f}\n')

Gaussian Process Classification Test Set Accuracy: 0.9811

Logistic Regression Test set accuracy: 0.6071

Optimal (3) NN Test set accuracy: 0.8435



In [93]:
print('Gaussian Process: \n', classification_report(y_test, gp_y_pred))
print('Logistic Regression: \n', classification_report(y_test, lg_y_pred))
print('k-NN Classification: \n', classification_report(y_test, knn_y_pred))

Gaussian Process: 
               precision    recall  f1-score   support

         0.0       1.00      0.96      0.98       597
         1.0       0.96      1.00      0.98       566

    accuracy                           0.98      1163
   macro avg       0.98      0.98      0.98      1163
weighted avg       0.98      0.98      0.98      1163

Logistic Regression: 
               precision    recall  f1-score   support

         0.0       0.66      0.49      0.56       597
         1.0       0.58      0.73      0.64       566

    accuracy                           0.61      1163
   macro avg       0.62      0.61      0.60      1163
weighted avg       0.62      0.61      0.60      1163

k-NN Classification: 
               precision    recall  f1-score   support

         0.0       1.00      0.70      0.82       597
         1.0       0.76      1.00      0.86       566

    accuracy                           0.84      1163
   macro avg       0.88      0.85      0.84      1163
weighted